In [1]:
# Section must be included at the beginning of each new notebook. Remember to change the app name.
# If you're using VirtualBox, change the below to '/home/user/spark-2.1.1-bin-hadoop2.7'
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('linear_regression_docs').getOrCreate()

# If you're getting an error with numpy, please type 'sudo pip3 install numpy --user' into the console.
# If you're getting an error with another package, type 'sudo pip3 install PACKAGENAME --user'. 
# Replace PACKAGENAME with the relevant package (such as pandas, etc).
from pyspark.ml.regression import LinearRegression

In [3]:
# Load model training data. Location of the data may be different.
training = spark.read.format("csv").load("Datasets/forestfires.csv")

In [4]:
training.show()

+---+---+-----+---+----+-----+-----+----+----+---+----+----+----+
|_c0|_c1|  _c2|_c3| _c4|  _c5|  _c6| _c7| _c8|_c9|_c10|_c11|_c12|
+---+---+-----+---+----+-----+-----+----+----+---+----+----+----+
|  X|  Y|month|day|FFMC|  DMC|   DC| ISI|temp| RH|wind|rain|area|
|  7|  5|  mar|fri|86.2| 26.2| 94.3| 5.1| 8.2| 51| 6.7|   0|   0|
|  7|  4|  oct|tue|90.6| 35.4|669.1| 6.7|  18| 33| 0.9|   0|   0|
|  7|  4|  oct|sat|90.6| 43.7|686.9| 6.7|14.6| 33| 1.3|   0|   0|
|  8|  6|  mar|fri|91.7| 33.3| 77.5|   9| 8.3| 97|   4| 0.2|   0|
|  8|  6|  mar|sun|89.3| 51.3|102.2| 9.6|11.4| 99| 1.8|   0|   0|
|  8|  6|  aug|sun|92.3| 85.3|  488|14.7|22.2| 29| 5.4|   0|   0|
|  8|  6|  aug|mon|92.3| 88.9|495.6| 8.5|24.1| 27| 3.1|   0|   0|
|  8|  6|  aug|mon|91.5|145.4|608.2|10.7|   8| 86| 2.2|   0|   0|
|  8|  6|  sep|tue|  91|129.5|692.6|   7|13.1| 63| 5.4|   0|   0|
|  7|  5|  sep|sat|92.5|   88|698.6| 7.1|22.8| 40|   4|   0|   0|
|  7|  5|  sep|sat|92.5|   88|698.6| 7.1|17.8| 51| 7.2|   0|   0|
|  7|  5| 